In [2]:
# Import packages
import numpy as np
import random
import matplotlib.pyplot as plt
import math

In [3]:
# Classic game in 2D

class Board():
  def __init__(self, rows=4, cols=4, prob_2=0.9, finish_value=2048):
      self.board = np.zeros((rows, cols), dtype=int)
      self.prob_2 = prob_2
      self.add_2_4()
      self.finish_value = finish_value # reaching 2048 is stopping criterion
      self.game_over = False
      self.score = 0

  def reset(self):
      self.__init__()
  
  # Randomly spawn a 2 or 4 into the board
  def add_2_4(self):
      # Looking for open spaces
      open_row, open_col = (self.board == 0).nonzero()
      if open_row.size > 0:
          new_pos = random.randint(0, open_row.size - 1) # new position
          self.board[open_row[new_pos], open_col[new_pos]] = 2 + 2 * (random.random() > self.prob_2) # 2 if rand < prob(2), otherwise 4
          self.board = self.board.astype(int)
        
  # Move tiles to the left, combine if similar
  def tiles_left(self, column):

    no_zeros = column[np.nonzero(column)[0]]
    col = np.append(np.zeros(1), no_zeros)

    for i in range(1, len(col)):
        if col[i] == col[i - 1]:
          col[i - 1] = int(2) * col[i - 1]
          col[i] = 0

          # Combining increases score
          self.score += col[i - 1]

    output_nonzeros = col[np.nonzero(col)] # non-zero values
    moved_zeros = np.zeros(len(column) - len(output_nonzeros)) # tail of 0s
    moved = np.append(output_nonzeros, moved_zeros) # concatenate

    return moved


  # Take input action, swiping in specified direction
  def action(self, direction):
    # 0: left, 1: up, 2: right, 3: down
    if 'A' in str.upper(direction): d = 0
    if 'W' in str.upper(direction): d = 1
    if 'D' in str.upper(direction): d = 2
    if 'S' in str.upper(direction): d = 3
    
    rotated_board = np.rot90(self.board, d)
    cols = [rotated_board[i, :] for i in range(rotated_board.shape[0])]
    new_board = np.array([self.tiles_left(c) for c in cols])
    return np.rot90(new_board, -d)
  
  # Game over if no more valid moves to play
  def check_game_over(self):
    if np.any(self.board == self.finish_value): # stop if tile hits finish value
        return True
    
    if (self.board == 0).nonzero()[0].size > 0: # there are open spaces
        return False
    
    # No repeats in row or column
    for i in range(self.board.shape[0]):
      for j in range(self.board.shape[1]):
        if i != 0 and self.board[i - 1][j] == self.board[i][j]:
          return False
        if j != 0 and self.board[i][j - 1] == self.board[i][j]:
          return False
        
    return True # otherwise, game is finished
  
  
  # Combine the individual actions together
  def step(self, direction):
    new_board = self.action(direction)
    if not (new_board == self.board).all():
      self.board = new_board.astype(int)
      self.add_2_4()

In [ ]:
# To play 2048 in 2D
game = Board(rows = 4, cols = 4, prob_2 = 0.9, finish_value = 2048)
print(game.board, '\n-------------\n')
finish = False

while not finish:
    direction = input('left (A) | up (W) | right (D) | down (S)')
    game.step(direction)
    finish = game.check_game_over()
    print(game.board, '\n-------------\n')

print('Final score: {}'.format(game.score))

In [4]:
# Game in 3D

class Board_3D():
  def __init__(self, rows=3, cols=3, pipes=3, prob_2=0.9, finish_value=2048):
      self.board = np.zeros((rows, cols, pipes), dtype=int)
      self.prob_2 = prob_2
      self.add_2_4()
      self.finish_value = finish_value # reaching 2048 is stopping criterion
      self.game_over = False
      self.score = 0

  def reset(self):
      self.__init__()
  
  # Randomly spawn a 2 or 4 into the board
  def add_2_4(self):
      # Looking for open spaces
      open_row, open_col, open_pipe = (self.board == 0).nonzero()
      if open_row.size > 0:
          new_pos = random.randint(0, open_row.size - 1) # new position
          self.board[open_row[new_pos], open_col[new_pos], open_pipe[new_pos]] = 2 + 2 * (random.random() > self.prob_2) # 2 if rand < prob(2), otherwise 4
          self.board = self.board.astype(int)
        
  # Move tiles to the left, combine if similar
  def tiles_left(self, column):

    no_zeros = column[np.nonzero(column)[0]]
    col = np.append(np.zeros(1), no_zeros)

    for i in range(1, len(col)):
        if col[i] == col[i - 1]:
          col[i - 1] = int(2) * col[i - 1]
          col[i] = 0

          # Combining increases score
          self.score += col[i - 1]

    output_nonzeros = col[np.nonzero(col)] # non-zero values
    moved_zeros = np.zeros(len(column) - len(output_nonzeros)) # tail of 0s
    moved = np.append(output_nonzeros, moved_zeros) # concatenate

    return moved


  # Take input action, swiping in specified direction
  def action(self, direction):
    # (Q) away, (W) up, (E) toward
    # (A) left, (S) down, (D) right
    
    if 'A' in str.upper(direction): d1 = 0; ax1 = (1, 2); d2 = 0; ax2 = (2, 1)
    if 'S' in str.upper(direction): d1 = 1; ax1 = (2, 1); d2 = 0; ax2 = (2, 1)
    if 'D' in str.upper(direction): d1 = 2; ax1 = (1, 2); d2 = 0; ax2 = (2, 1)
    if 'W' in str.upper(direction): d1 = 1; ax1 = (1, 2); d2 = 0; ax2 = (2, 1)
    if 'Q' in str.upper(direction): d1 = 1; ax1 = (1, 0); d2 = 1; ax2 = (1, 2)
    if 'E' in str.upper(direction): d1 = 1; ax1 = (0, 1); d2 = 1; ax2 = (1, 2)

    # Rotate appropriately
    rotated_board = np.rot90(np.rot90(self.board, d1, axes=ax1), d2, axes=ax2)

    # Collapse board into 2 dimensions
    rotated_board_reshaped = rotated_board.reshape(-1, rotated_board.shape[2])

    # Apply transformations
    cols = [rotated_board_reshaped[i, :] for i in range(rotated_board_reshaped.shape[0])]
    new_board = np.array([self.tiles_left(c) for c in cols])

    # Set back into 3 dimensions
    new_board_reshaped = new_board.reshape(rotated_board.shape)

    # Undo rotation
    return np.rot90(np.rot90(new_board_reshaped, -d2, axes=ax2), -d1, axes=ax1)
  
  # Game over if no more valid moves to play
  def check_game_over(self):
    if np.any(self.board == self.finish_value): # stop if tile hits finish value
        return True
    
    if (self.board == 0).nonzero()[0].size > 0: # there are open spaces
        return False
    
    # No repeats in row or column
    for i in range(self.board.shape[0]):
      for j in range(self.board.shape[1]):
        if i != 0 and self.board[i - 1][j] == self.board[i][j]:
          return False
        if j != 0 and self.board[i][j - 1] == self.board[i][j]:
          return False
        
    return True # otherwise, game is finished
  
  
  # Combine the individual actions together
  def step(self, direction):
    new_board = self.action(direction)
    if not (new_board == self.board).all():
      self.board = new_board.astype(int)
      self.add_2_4()

In [ ]:
# To play 2048 in 3D

game2 = Board_3D(rows = 3, cols = 3, pipes = 3, prob_2 = 0.9, finish_value = 2048)
print(game2.board, '\n---------------\n')
finish2 = False

while not finish2:
    direction = input('(Q) away | (W) up | (E) toward | (A) left | (S) down | (D) right')
    game2.step(direction)
    print(game2.board, '\n---------------\n')
    finish2 = game2.check_game_over()

print('Final score: {}'.format(game2.score))